Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name below:

In [ ]:
let name = "sathvick"
let rollno = "cs19b010"

## Important notes about grading:

1. **Compiler errors:** All code you submit must compile. Programs that do not compile will probably receive an automatic zero. If you are having trouble getting your assignment to compile, please visit consulting hours. If you run out of time, it is better to comment out the parts that do not compile, than hand in a more complete file that does not compile.
2. **Late assignments:** Please carefully review the course website's policy on late assignments, as all assignments handed in after the deadline will be considered late. Verify on moodle that you have submitted the correct version, before the deadline. Submitting the incorrect version before the deadline and realizing that you have done so after the deadline will be counted as a late submission.

# Lambda Calculus Interpreter

In this assignment, you will implement lambda calculus interpreters that use different reduction strategies. The abstract syntax tree (AST) for lambda expressions is the one that we have seen in class:


```ocaml
type expr = 
  | Var of string
  | Lam of string * expr
  | App of expr * expr
```


You are provided a parser function `parse_string` that converts a string to this AST and a printer function `string_of_expr` that converts the AST to string. For example, 

In [75]:
#use "init.ml"
open Syntax
let parse_string = Lambda_parse.parse_string
let string_of_expr = string_of_expr

let _ = parse_string "(\\x.x) (\\y.y)"
let _ = string_of_expr (App (Var "x",Lam("y",App(Var "y", Var "x"))))

val alpha_equiv : Syntax.expr -> Syntax.expr -> bool = <fun>


val parse_string : string -> Syntax.expr = <fun>


val string_of_expr : Syntax.expr -> string = <fun>


- : Syntax.expr = App (Lam ("x", Var "x"), Lam ("y", Var "y"))


- : string = "x (λy.y x)"


You will need some helper functions to operate over sets. Since we have not studied set data structure in OCaml. We will use lists instead and implement set functionality on top of lists. You can use the functions from the OCaml list standard library for this assignment. The documentation for OCaml list module is available in the [OCaml manual](https://caml.inria.fr/pub/docs/manual-ocaml/libref/List.html). 

## Problem 1

Implement a function

```ocaml
mem : 'a -> 'a list -> bool
```

`mem e l` returns `true` if the element `e` is present in the list. Otherwise, it returns false. 

In [54]:
let rec mem e l v  = 
match l with
|[]->v
|h::t->if(h = e) then true else (mem e t v)
let mem e l = mem e l false

val mem : 'a -> 'a list -> bool -> bool = <fun>


val mem : 'a -> 'a list -> bool = <fun>


In [55]:
(* 5 points *)
assert (mem "b" ["a";"b";"c"] = true);
assert (mem "x" ["a";"b";"c"] = false)

- : unit = ()


## Problem 2

Implement a function

```ocaml
remove : 'a -> 'a list -> 'a list
```

`remove e l` returns a list `l'` with all the element in `l` except `e`. `remove` also preserves the order of the elements not removed. If `e` is not present in `l`, then return `l`.

In [56]:
let rec remove e l = 
match l with
|[]->[]
|h::t->if(h=e) then (remove e t) else h::(remove e t)   

val remove : 'a -> 'a list -> 'a list = <fun>


In [57]:
(* 5 points *)
assert (remove "b" ["a";"b";"c"] = ["a";"c"]);
assert (remove "x" ["a";"b";"c"] = ["a";"b";"c"])

- : unit = ()


## Problem 3

Implement a function

```ocaml
union : string list -> string list -> string list
```

`union l1 l2` performs set union of elements in `l1` and `l2`. The elements in the result list `l` must be lexicographically sorted. Hint: You may want to use the functions `List.sort` and `remove_stutter` from assignment 1 to implement union. Here is an example of using `List.sort`. 

In [58]:
assert (List.sort String.compare ["x";"a";"b";"m"] = ["a";"b";"m";"x"])

- : unit = ()


In [59]:
let rec remove_stutter (l,a)=
match l with
|[]->[]
|hd::tl -> 
match a with Some(temp)->
if(hd=temp) then remove_stutter (tl,Some(hd))
else hd::remove_stutter (tl,Some(hd))
|None->hd::remove_stutter (tl,Some(hd))
let remove_stutter l =remove_stutter(l,None)

let rec comb1 l1 l2 =
match l1 with 
|[]->l2
|h::t-> comb1 t (h::l2)


let rec combine l1 l2 l3 =
match l1 with
|[]-> comb1 l2 l3
|h::t-> combine t l2 (h::l3)


let union l1 l2 = 
 remove_stutter(List.sort compare (combine l1 l2 []))


val remove_stutter : 'a list * 'a option -> 'a list = <fun>


val remove_stutter : 'a list -> 'a list = <fun>


val comb1 : 'a list -> 'a list -> 'a list = <fun>


val combine : 'a list -> 'a list -> 'a list -> 'a list = <fun>


val union : 'a list -> 'a list -> 'a list = <fun>


In [60]:
(* 5 points *)
assert (union ["a"; "c"; "b"] ["d"; "b"; "x"; "a"] = ["a"; "b"; "c"; "d"; "x"])

- : unit = ()


## Problem 4

Implement a function

```ocaml
add : 'a -> 'a list -> 'a list
```

`add e l` does a set addition of element `e` to list `l` and returns a list. The resultant list is sorted. 

In [61]:
let rec check e l=
match l with
|[]->false
|h::t->if h=e then true else check e t
let add e l = 
 if(check e l) then List.sort compare(l) else List.sort compare(e::l)

val check : 'a -> 'a list -> bool = <fun>


val add : 'a -> 'a list -> 'a list = <fun>


In [62]:
add "b" ["a";"c"] 

- : string list = ["a"; "b"; "c"]


In [63]:
(* 5 points *)
assert (add "b" ["a";"c"] = ["a";"b";"c"]);
assert (add "a" ["c"; "a"] = ["a";"c"])

- : unit = ()


## Substitution

At the heart of reducing lambda expressions is substitution. Recall from the lecture that substitution requires us to generate fresh variable names that is different from every other name used in the current context. We will use the following helper function to generate fresh names.

In [64]:
let r = ref 0                                                                       
                                                                                    
let fresh s =                                                                       
  let v = !r in                                                                     
  r := !r + 1;                                                                      
  s ^ (string_of_int v)

val r : int ref = {contents = 0}


val fresh : string -> string = <fun>


It uses mutability features of OCaml which we will study in later lectures. You can use the `fresh` function as follows:

In [65]:
let a = fresh "a"
let b = fresh "b"

val a : string = "a0"


val b : string = "b1"


## Problem 5

Implement a function 

```ocaml
free_variables: expr -> string list
```

that returns the free variables in the given lambda term.

In [80]:
let rec remove e l = 
match l with
|[]->[]
|h::t->if(h=e) then (remove e t) else h::(remove e t)  

let rec free_variables e =
match e with
|Var x-> [x]
|Lam (x,t)->remove x (free_variables t)
|App (x,y)-> union (free_variables x)  (free_variables y) 


val remove : 'a -> 'a list -> 'a list = <fun>


val free_variables : Syntax.expr -> string list = <fun>


In [81]:
(* 20 points *)
assert (free_variables (parse_string "\\x.x") = []);
assert (free_variables (parse_string "\\x.y") = ["y"])

- : unit = ()


## Problem 6

Implement the function

```ocaml
substitute : expr -> string -> expr -> expr
```

where `substitute e x v` does `e[v/x]`. For renaming `x` in `Lam(x,y)` with a fresh name, use `Lam (fresh x, ...)`.

In [99]:
let rec check lis y =
match lis with
|[]->false
|h::t->if(h=y) then true else check t y

let rec substitute expr a b =
match expr with
|Var x ->if(x=a) then b else expr
|Lam (x,t)-> if(x=a) then expr else if(check (free_variables b) x=false) then Lam (x,substitute t a b) 
 else Lam((fresh x),substitute (substitute t x (Var(fresh x))) a b)
|App(x ,y)-> App(substitute x a b,substitute y a b) 

val check : 'a list -> 'a -> bool = <fun>


val substitute : Syntax.expr -> string -> Syntax.expr -> Syntax.expr = <fun>


In [100]:
(* 20 points *)
assert (alpha_equiv 
          (substitute (parse_string "\\y.x") "x" (parse_string "\\z.z w")) 
          (parse_string "λy.λz.z w"));
assert (alpha_equiv 
          (substitute (parse_string "\\x.x") "x" (parse_string "y"))
          (parse_string "λx.x"));
assert (alpha_equiv 
          (substitute (parse_string "\\x.y") "y" (parse_string "x"))
          (parse_string "λx0.x"))

- : unit = ()


## Problem 7

Implement a single step of the call-by-value reduction. Implement the function

```ocaml
reduce_cbv : expr -> expr option
```

which does a single step of the call-by-value reduction. Recall that call-by-value reduction is determinisitic. Hence, if reduction is possible, then a single rule applies. `reduce e` returns `Some e'` if reduction is possible and `e'` is the new expression. `reduce e` returns `None` if reduction is not possible.

In [101]:
let rec reduce_cbv e =
begin match e with 
|Var x -> None
|Lam (x,t)-> None
|App (m,n)-> begin match (reduce_cbv m) with
             |None->begin match (reduce_cbv n) with
                    |None->begin match m with
                           |Var x->None
                           |Lam(y,t1)->Some (substitute t1 y n)
                           |App(p,q)->None end;
                    |Some expr1->Some (App(m,expr1))end;
              |Some expr2->Some (App(expr2,n)) end; 
end;              

val reduce_cbv : Syntax.expr -> Syntax.expr option = <fun>


In [102]:
(* 20 points *)
begin match reduce_cbv (parse_string "(\\x.x) ((\\x.x) (\\z.(\\x.x) z))") with
| Some expr -> assert (alpha_equiv expr (parse_string "(λx.x) (λz.(λx.x) z)"))
| None -> assert false
end;
        
begin match reduce_cbv (parse_string "(λx.x) (λz.(λx.x) z)") with
| Some expr -> assert (alpha_equiv expr (parse_string "λz.(λx.x) z"))
| None -> assert false
end;
        
assert (reduce_cbv (parse_string "λz.(λx.x) z") = None);
        
begin match reduce_cbv (parse_string "(λx.y) ((λx.x x) (λx.x x))") with
| Some expr -> assert (alpha_equiv expr (parse_string "(λx.y) ((λx.x x) (λx.x x))"))
| None -> assert false
end;

assert (reduce_cbv (parse_string "x y z") = None)


- : unit = ()


## Problem 8

Implement a single step of the call-by-name reduction. Implement the function

```ocaml
reduce_cbn : expr -> expr option
```

The rest of the instructions are same as `reduce_cbv`.

In [107]:
let rec reduce_cbn e =
match e with
|Var x -> None
|Lam (x,t)->None
|App (m,n)-> match (reduce_cbn m) with
             |Some expr ->Some (App (expr,n))
             |None->match m with
                    |Lam (y,t1)->Some (substitute t1 y n)
                    |_->None

val reduce_cbn : Syntax.expr -> Syntax.expr option = <fun>


In [108]:
(* 20 points *)
begin match reduce_cbn (parse_string "(\\x.x) ((\\x.x) (\\z.(\\x.x) z))") with
| Some expr -> assert (alpha_equiv expr (parse_string "(λx.x) (λz.(λx.x) z)"))
| None -> assert false
end;
        
begin match reduce_cbn (parse_string "(λx.x) (λz.(λx.x) z)") with
| Some expr -> assert (alpha_equiv expr (parse_string "λz.(λx.x) z"))
| None -> assert false
end;
        
assert (reduce_cbn (parse_string "λz.(λx.x) z") = None);
        
begin match reduce_cbn (parse_string "(λx.y) ((λx.x x) (λx.x x))") with
| Some expr -> assert (alpha_equiv expr (parse_string "y"))
| None -> assert false
end;

begin match reduce_cbn (parse_string "(\\x.x x) ((\\z.z) y)") with
| Some expr -> assert (alpha_equiv expr (parse_string "(λz.z) y ((λz.z) y)"))
| None -> assert false
end;

assert (reduce_cbn (parse_string "x y z") = None)


- : unit = ()


## Problem 9

Implement a single step of the normal order reduction. Implement the function

```ocaml
reduce_normal : expr -> expr option
```

The rest of the instructions are same as `reduce_cbv`.

In [112]:
let rec reduce_normal e =
match e with
|Var x -> None

|Lam (x,t)->(match (reduce_normal t) with
            |Some expr ->Some (Lam (x,expr))
            |_ ->None)  
|App (m,n)->match m with
            |Lam (y,t1)->Some (substitute t1 y n)
            |_->match (reduce_normal m) with
                |Some expr->Some (App(expr,n))
                |_->(match (reduce_normal n) with
                    |Some expr1->Some (App(m,expr1))
                    |_->None)

val reduce_normal : Syntax.expr -> Syntax.expr option = <fun>


In [113]:
(* 20 points *)
begin match reduce_normal (parse_string "(\\x.x) ((\\x.x) (\\z.(\\x.x) z))") with
| Some expr -> assert (alpha_equiv expr (parse_string "(λx.x) (λz.(λx.x) z)"))
| None -> assert false
end;
        
begin match reduce_normal (parse_string "(λx.x) (λz.(λx.x) z)") with
| Some expr -> assert (alpha_equiv expr (parse_string "λz.(λx.x) z"))
| None -> assert false
end;
        
begin match reduce_normal (parse_string "λz.(λx.x) z") with
| Some expr -> assert (alpha_equiv expr (parse_string "λz. z"))
| None -> assert false
end;
        
begin match reduce_normal (parse_string "(λx.y) ((λx.x x) (λx.x x))") with
| Some expr -> assert (alpha_equiv expr (parse_string "y"))
| None -> assert false
end;

begin match reduce_normal (parse_string "(\\x.x x) ((\\z.z) y)") with
| Some expr -> assert (alpha_equiv expr (parse_string "(λz.z) y ((λz.z) y)"))
| None -> assert false
end;
        
begin match reduce_normal (parse_string "f (\\x.x x) ((\\z.z) y)") with
| Some expr -> assert (alpha_equiv expr (parse_string "f (λx.x x) y"))
| None -> assert false
end;

begin match reduce_normal (parse_string "(\\x.(\\z.z) y) (\\x.x x)") with
| Some expr -> assert (alpha_equiv expr (parse_string "(λz.z) y"))
| None -> assert false
end


- : unit = ()


In [114]:
let rec eval log depth reduce expr =                     
  if depth = 0 then failwith "non-termination?"                                  
  else                                                                     
    begin match reduce expr with
    | None -> expr
    | Some expr' ->
      if log then print_endline ("= " ^ (string_of_expr expr'));                 
      eval log (depth-1) reduce expr'                                    
    end
  
let eval_cbv = eval true 1000 reduce_cbv
let eval_cbn = eval true 1000 reduce_cbn
let eval_normal = eval true 1000 reduce_normal

val eval :
  bool ->
  int -> (Syntax.expr -> Syntax.expr option) -> Syntax.expr -> Syntax.expr =
  <fun>


val eval_cbv : Syntax.expr -> Syntax.expr = <fun>


val eval_cbn : Syntax.expr -> Syntax.expr = <fun>


val eval_normal : Syntax.expr -> Syntax.expr = <fun>


In [115]:
(* 10 points *)
let zero = parse_string "\\f.\\x. x" in
let one = parse_string "\\f.\\x. f x" in
let two = parse_string "\\f.\\x. f (f x)" in
let three = parse_string "\\f.\\x. f (f (f x))" in

let plus = parse_string "λm. λn. λs. λz. m s (n s z)" in
let mult = parse_string "λm. λn. λs. λz. m (n s) z" in

assert (alpha_equiv (eval_normal (App (App (plus, one), two))) three);
print_endline "";
assert (alpha_equiv (eval_normal (App (App (mult, one), three))) three);
print_endline "";
assert (alpha_equiv (eval_normal (App (App (mult, zero), three))) zero)

= (λn.λs.λz.(λf.λx.f x) s (n s z)) (λf.λx.f (f x))
= λs.λz.(λf.λx.f x) s ((λf.λx.f (f x)) s z)
= λs.λz.(λx.s x) ((λf.λx.f (f x)) s z)
= λs.λz.s ((λf.λx.f (f x)) s z)
= λs.λz.s ((λx.s (s x)) z)
= λs.λz.s (s (s z))

= (λn.λs.λz.(λf.λx.f x) (n s) z) (λf.λx.f (f (f x)))
= λs.λz.(λf.λx.f x) ((λf.λx.f (f (f x))) s) z
= λs.λz.(λx.(λf.λx.f (f (f x))) s x) z
= λs.λz.(λf.λx.f (f (f x))) s z
= λs.λz.(λx.s (s (s x))) z
= λs.λz.s (s (s z))

= (λn.λs.λz.(λf.λx.x) (n s) z) (λf.λx.f (f (f x)))
= λs.λz.(λf.λx.x) ((λf.λx.f (f (f x))) s) z
= λs.λz.(λx.x) z
= λs.λz.z


- : unit = ()
